In [1]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())

PyTorch version: 2.5.1+cu124
Torchvision version: 0.20.1+cu124
CUDA is available: False


/home/zhongyi/anaconda3/envs/sam/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )
np.random.seed(3)


using device: cpu


First, load the SAM 2 model and predictor. Change the path below to point to the SAM 2 checkpoint. Running on CUDA and using the default model are recommended for best results.

In [3]:
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

sam2_checkpoint = "../checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

sam2_model = build_sam2(model_cfg, sam2_checkpoint, device=device)

predictor = SAM2ImagePredictor(sam2_model)

In [4]:
import numpy as np
from scipy.ndimage import label, find_objects, binary_erosion, binary_dilation
from scipy.ndimage import generate_binary_structure


def get_largest_connected_component_and_bbox_mask(mask, radius=5, dilation_radius=20):
    # Define a structuring element for erosion and dilation
    struct_elem = generate_binary_structure(2, 1)
    
    # Erode and then dilate the mask to remove small noise
    mask_eroded = binary_erosion(mask, structure=struct_elem, iterations=radius)
    mask_cleaned = binary_dilation(mask_eroded, structure=struct_elem, iterations=radius)
    
    # Label connected components in the cleaned mask
    labeled_array, num_features = label(mask_cleaned)
    
    # Get sizes of connected components
    component_sizes = [(labeled_array == i).sum() for i in range(1, num_features + 1)]
    
    # Identify the largest component by its label
    largest_component_label = np.argmax(component_sizes) + 1
    
    # Create a mask with only the largest connected component
    largest_component = (labeled_array == largest_component_label).astype(np.uint8)
    
    # Find the bounding box of the largest connected component
    bbox = find_objects(labeled_array == largest_component_label)[0]
    min_row, min_col = bbox[0].start, bbox[1].start
    max_row, max_col = bbox[0].stop, bbox[1].stop
    
    # Create an empty array of the same shape as the input mask
    bbox_mask = np.zeros_like(mask, dtype=np.uint8)
    
    # Set the bounding box region to 1
    bbox_mask[min_row:max_row, min_col:max_col] = 1

    # Dilate the bounding box mask by 20 pixels
    bbox_mask_dilated = binary_dilation(bbox_mask, structure=struct_elem, iterations=dilation_radius)
    
    return largest_component, bbox_mask_dilated


def crop_image_by_bbox_per_channel(image, bbox_mask):
    # Find the coordinates of the bounding box from the bbox mask
    rows = np.any(bbox_mask, axis=1)
    cols = np.any(bbox_mask, axis=0)
    min_row, max_row = np.where(rows)[0][[0, -1]]
    min_col, max_col = np.where(cols)[0][[0, -1]]
    
    # Crop each channel of the image using the bounding box coordinates
    cropped_channels = [image[min_row:max_row+1, min_col:max_col+1, c] for c in range(image.shape[2])]
    
    # Stack cropped channels back together to form the final cropped image
    cropped_image = np.stack(cropped_channels, axis=-1)
    
    return cropped_image

def get_center_and_center_rectangle(array, size_factor=0.2):
    h, w = array.shape
    cy, cx = h // 2, w // 2
    rh, rw = int(h * size_factor), int(w * size_factor)
    
    # Calculate the corner points
    top_left = [cx - rw // 2, cy - rh // 2]
    top_right = [top_left[0], top_left[1] + rh]
    bottom_left = [top_left[0] + rw, top_left[1]]
    bottom_right = [top_left[0] + rw, top_left[1] + rh]
    
    # Return the coordinates as a list
    return np.array([[cx, cy], top_left, top_right, bottom_left, bottom_right])



## Example image

In [5]:
from PIL import Image

# Directories for input images and output cropped images
img_dir  = 'quarter_reslution_tactile_img_717/'  # Input directory containing raw tactile images
crop_dir = 'quarter_reslution_tactile_img_cropped_1point/'  # Output directory for cropped images

# Iterate through all files in the input directory in sorted order
for idx, d in enumerate(sorted(os.listdir(img_dir))):
    # Construct the full path to the input image
    img_path = img_dir + d    
    
    # Modify the file name to include '_cropped' before the extension ('id001.jpg' -> 'id001_cropped.jpg')
    dot_index = d.rfind('.')
    crop_d = d[:dot_index] + '_cropped' + d[dot_index:]
    targ_path = crop_dir + crop_d  # Full path for the cropped image output
    
    # Log the index and file name being processed
    print(idx, d)
    
    # Open the input image and ensure it's in RGB format
    image = Image.open(img_path)
    image = np.array(image.convert("RGB"))
    
    # Set the image for the predictor model
    predictor.set_image(image)
    
    # Define a central point as the input for the model
    input_point = np.array([[int(image.shape[1]/2), int(image.shape[0]/2)]])  # Central point of the image
    input_label = np.array([1])  # Label for the input point (e.g., foreground)
    
    # Perform prediction using the SAM2 model
    masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True,  # Generate multiple masks
    )
    
    # Sort the masks by their confidence scores in descending order
    sorted_ind = np.argsort(scores)[::-1]
    masks = masks[sorted_ind]  # Reorder masks by scores
    scores = scores[sorted_ind]  # Reorder scores
    logits = logits[sorted_ind]  # Reorder logits
    
    # Extract the largest connected component and the bounding box mask from the highest-scoring mask
    largest_component, bbox_mask = get_largest_connected_component_and_bbox_mask(masks[0])
    
    # Crop the image using the bounding box mask
    cropped_image = crop_image_by_bbox_per_channel(image, bbox_mask)
    
    # Ensure the cropped image is in the correct data type for saving
    cropped_image = (cropped_image * 255).astype(np.uint8) if cropped_image.dtype != np.uint8 else cropped_image
    
    # Save the cropped image to the target path
    Image.fromarray(cropped_image).save(targ_path)
    
    # Log the success of the cropping operation
    print(idx, d, "Cropped image shape:", cropped_image.shape, 'saved', targ_path)


0 ID_001_P1099730.JPG
0 ID_001_P1099730.JPG Cropped image shape: (2339, 1793, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_001_P1099730_cropped.JPG
1 ID_001_P1099731.JPG
1 ID_001_P1099731.JPG Cropped image shape: (2341, 1787, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_001_P1099731_cropped.JPG
2 ID_001_P1099732.JPG
2 ID_001_P1099732.JPG Cropped image shape: (429, 124, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_001_P1099732_cropped.JPG
3 ID_001_P1099734.JPG
3 ID_001_P1099734.JPG Cropped image shape: (2337, 1788, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_001_P1099734_cropped.JPG
4 ID_001_P1099735.JPG
4 ID_001_P1099735.JPG Cropped image shape: (2332, 1786, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_001_P1099735_cropped.JPG
5 ID_001_P1099736.JPG
5 ID_001_P1099736.JPG Cropped image shape: (2334, 1792, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_001_P1099736_cropped.JPG
6 ID_001_P1099737.JPG
6 ID_001_P1099737.JP

51 ID_010_IMG_6352.jpeg Cropped image shape: (2667, 3682, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_010_IMG_6352_cropped.jpeg
52 ID_010_IMG_6353.jpeg
52 ID_010_IMG_6353.jpeg Cropped image shape: (2704, 3671, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_010_IMG_6353_cropped.jpeg
53 ID_010_IMG_6354.jpeg
53 ID_010_IMG_6354.jpeg Cropped image shape: (2472, 3684, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_010_IMG_6354_cropped.jpeg
54 ID_010_IMG_6355.jpeg
54 ID_010_IMG_6355.jpeg Cropped image shape: (2632, 3663, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_010_IMG_6355_cropped.jpeg
55 ID_012_P1099772.JPG
55 ID_012_P1099772.JPG Cropped image shape: (2213, 1659, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_012_P1099772_cropped.JPG
56 ID_012_P1099773.JPG
56 ID_012_P1099773.JPG Cropped image shape: (2224, 1665, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_012_P1099773_cropped.JPG
57 ID_012_P1099774.JPG
57 ID_012_P109977

102 ID_017_P1099851.JPG Cropped image shape: (264, 86, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_017_P1099851_cropped.JPG
103 ID_017_P1099852.JPG
103 ID_017_P1099852.JPG Cropped image shape: (1081, 124, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_017_P1099852_cropped.JPG
104 ID_017_P1099854.JPG
104 ID_017_P1099854.JPG Cropped image shape: (2049, 1472, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_017_P1099854_cropped.JPG
105 ID_017_P1099855.JPG
105 ID_017_P1099855.JPG Cropped image shape: (2057, 1474, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_017_P1099855_cropped.JPG
106 ID_017_P1099856.JPG
106 ID_017_P1099856.JPG Cropped image shape: (2058, 1476, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_017_P1099856_cropped.JPG
107 ID_017_P1099857.JPG
107 ID_017_P1099857.JPG Cropped image shape: (2057, 1474, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_017_P1099857_cropped.JPG
108 ID_017_P1099858.JPG
108 ID_017_P1099858.

152 ID_022_P1099870.JPG Cropped image shape: (2065, 1465, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_022_P1099870_cropped.JPG
153 ID_022_P1099871.JPG
153 ID_022_P1099871.JPG Cropped image shape: (2062, 1463, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_022_P1099871_cropped.JPG
154 ID_023_P1099967.JPG
154 ID_023_P1099967.JPG Cropped image shape: (2050, 1449, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_023_P1099967_cropped.JPG
155 ID_023_P1099968.JPG
155 ID_023_P1099968.JPG Cropped image shape: (2042, 1458, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_023_P1099968_cropped.JPG
156 ID_023_P1099969.JPG
156 ID_023_P1099969.JPG Cropped image shape: (2030, 1457, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_023_P1099969_cropped.JPG
157 ID_023_P1099970.JPG
157 ID_023_P1099970.JPG Cropped image shape: (2035, 1457, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_023_P1099970_cropped.JPG
158 ID_023_P1099971.JPG
158 ID_023_P1099

202 ID_028_P1099903.JPG Cropped image shape: (2058, 1467, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_028_P1099903_cropped.JPG
203 ID_028_P1099904.JPG
203 ID_028_P1099904.JPG Cropped image shape: (2058, 1465, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_028_P1099904_cropped.JPG
204 ID_028_P1099905.JPG
204 ID_028_P1099905.JPG Cropped image shape: (2059, 1462, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_028_P1099905_cropped.JPG
205 ID_028_P1099906.JPG
205 ID_028_P1099906.JPG Cropped image shape: (2060, 1465, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_028_P1099906_cropped.JPG
206 ID_028_P1099907.JPG
206 ID_028_P1099907.JPG Cropped image shape: (2062, 1465, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_028_P1099907_cropped.JPG
207 ID_028_P1099908.JPG
207 ID_028_P1099908.JPG Cropped image shape: (2060, 1465, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_028_P1099908_cropped.JPG
208 ID_029_P1099911.JPG
208 ID_029_P1099

...

502 ID_075_P1100182.JPG Cropped image shape: (1860, 1312, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_075_P1100182_cropped.JPG
503 ID_076_P1100183.JPG
503 ID_076_P1100183.JPG Cropped image shape: (1858, 1311, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_076_P1100183_cropped.JPG
504 ID_076_P1100184.JPG
504 ID_076_P1100184.JPG Cropped image shape: (1858, 1306, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_076_P1100184_cropped.JPG
505 ID_076_P1100185.JPG
505 ID_076_P1100185.JPG Cropped image shape: (1859, 1311, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_076_P1100185_cropped.JPG
506 ID_076_P1100187.JPG
506 ID_076_P1100187.JPG Cropped image shape: (1858, 1306, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_076_P1100187_cropped.JPG
507 ID_076_P1100188.JPG
507 ID_076_P1100188.JPG Cropped image shape: (1858, 1309, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_076_P1100188_cropped.JPG
508 ID_076_P1100189.JPG
508 ID_076_P1100

552 ID_082_P1100236.JPG Cropped image shape: (1989, 1357, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_082_P1100236_cropped.JPG
553 ID_082_P1100237.JPG
553 ID_082_P1100237.JPG Cropped image shape: (1988, 1356, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_082_P1100237_cropped.JPG
554 ID_082_P1100238.JPG
554 ID_082_P1100238.JPG Cropped image shape: (1984, 1352, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_082_P1100238_cropped.JPG
555 ID_082_P1100239.JPG
555 ID_082_P1100239.JPG Cropped image shape: (1985, 1353, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_082_P1100239_cropped.JPG
556 ID_082_P1100240.JPG
556 ID_082_P1100240.JPG Cropped image shape: (1991, 1354, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_082_P1100240_cropped.JPG
557 ID_083_P1100241.JPG
557 ID_083_P1100241.JPG Cropped image shape: (1988, 1344, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_083_P1100241_cropped.JPG
558 ID_083_P1100242.JPG
558 ID_083_P1100

602 ID_088_P1100301.JPG Cropped image shape: (1942, 1288, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_088_P1100301_cropped.JPG
603 ID_088_P1100302.JPG
603 ID_088_P1100302.JPG Cropped image shape: (1943, 1288, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_088_P1100302_cropped.JPG
604 ID_088_P1100303.JPG
604 ID_088_P1100303.JPG Cropped image shape: (1943, 1285, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_088_P1100303_cropped.JPG
605 ID_088_P1100304.JPG
605 ID_088_P1100304.JPG Cropped image shape: (1943, 1287, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_088_P1100304_cropped.JPG
606 ID_088_P1100305.JPG
606 ID_088_P1100305.JPG Cropped image shape: (1940, 1287, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_088_P1100305_cropped.JPG
607 ID_088_P1100306.JPG
607 ID_088_P1100306.JPG Cropped image shape: (1940, 1287, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_088_P1100306_cropped.JPG
608 ID_088_P1100307.JPG
608 ID_088_P1100

652 ID_093_P1100364.JPG Cropped image shape: (1628, 1165, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_093_P1100364_cropped.JPG
653 ID_093_P1100365.JPG
653 ID_093_P1100365.JPG Cropped image shape: (1439, 78, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_093_P1100365_cropped.JPG
654 ID_093_P1100366.JPG
654 ID_093_P1100366.JPG Cropped image shape: (1305, 99, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_093_P1100366_cropped.JPG
655 ID_093_P1100367.JPG
655 ID_093_P1100367.JPG Cropped image shape: (1392, 138, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_093_P1100367_cropped.JPG
656 ID_093_P1100368.JPG
656 ID_093_P1100368.JPG Cropped image shape: (1282, 938, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_093_P1100368_cropped.JPG
657 ID_094_P1100369.JPG
657 ID_094_P1100369.JPG Cropped image shape: (895, 189, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_094_P1100369_cropped.JPG
658 ID_094_P1100370.JPG
658 ID_094_P1100370.JPG 

703 ID_101_P1100414.JPG Cropped image shape: (1614, 1182, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_101_P1100414_cropped.JPG
704 ID_101_P1100415.JPG
704 ID_101_P1100415.JPG Cropped image shape: (1660, 1325, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_101_P1100415_cropped.JPG
705 ID_101_P1100416.JPG
705 ID_101_P1100416.JPG Cropped image shape: (1583, 1111, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_101_P1100416_cropped.JPG
706 ID_101_P1100417.JPG
706 ID_101_P1100417.JPG Cropped image shape: (1655, 1179, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_101_P1100417_cropped.JPG
707 ID_101_P1100418.JPG
707 ID_101_P1100418.JPG Cropped image shape: (1585, 1115, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_101_P1100418_cropped.JPG
708 ID_102_P1100419.JPG
708 ID_102_P1100419.JPG Cropped image shape: (1580, 1116, 3) saved quarter_reslution_tactile_img_cropped_1point/ID_102_P1100419_cropped.JPG
709 ID_102_P1100420.JPG
709 ID_102_P1100